LLM acc 

In [1]:
import pandas as pd
from vllm import LLM, SamplingParams
import os

2024-10-23 15:31:43.227437: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-23 15:31:43.255577: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-23 15:31:43.759356: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import re
import json
import numpy as np
from sklearn.metrics import classification_report
import pandas as pd

In [3]:
# Function to clean and map Mistral's raw output to 'Yes' or 'No'
def clean_prediction(text):
    if not text or not isinstance(text, str):
        return None
    match = re.search(r'\b(Yes|No)\b', text)
    return match.group(0) if match else None

In [3]:
# Function to clean and map Mistral's raw output to 'Yes' or 'No'
def clean_prediction(text):
    if not text or not isinstance(text, str):
        return None
    match = re.search(r'\b(Yes|No)\b', text)
    return match.group(0) if match else None

In [6]:
def process_and_evaluate(test_data_path, output_csv_path):
    # Load the true labels and data from the .jsonl file
    data = []
    y_true = []
    with open(test_data_path, 'r') as f:
        for line in f:
            record = json.loads(line)
            data.append(record)  # Store each record for later
            y_true.append(record['target'])  # Assuming 'target' is the true label field

    # Initialize the Mistral LLM
    llm = LLM(model="/home/akazad/Mistral-7B-Instruct-v0.2-AWQ/", tensor_parallel_size=2, gpu_memory_utilization=0.8, enforce_eager=True, quantization="awq", dtype="auto")
    #llm = LLM(model="/home/akazad/Mistral-7B-Instruct-v0.2/", tensor_parallel_size=2, gpu_memory_utilization=0.8, enforce_eager=True, quantization="awq", dtype="auto")

    # Prepare the prompts
    prompt_template = ''' <s> [INST] You are an analytical tool specialized in processing and classifying GitHub commit messages. Your task is to assess the developer's intent in a given commit message, and categorize it into one of the following predefined categories based on its content:
                      
'Yes': A commit message that explicitly mention or demonstrate performance improvement or optimization, specifically in terms of execution time or resource utilization. The message  should clearly indicate actions that made the code run faster or more efficiently, use less memory, or more efficiently utilize system resources. This includes optimizations like replacing inefficient code patterns that are known to kill performance even if the message does not use the words 'improvement' or 'performance' explicitly. 

'No': A commit message  that do not pertain to performance enhancements. This includes changes related to testing, documentation, performance profiling/monitoring/debugging/analysis, and bug/error/crash fixes that don't explicitly mention performance improvement of the application itself. Also, code refactoring or feature addition without explicit performance optimization, and mentions of necessary, speculative, or potential performance enhancements without concrete evidence or results. Any message  that is irrelevant, unclear, ambiguous, or does not provide enough context to determine its intent should also be classified as 'No'. 

Additionally, pay close attention to the context in which terms like 'performance', 'improve', or 'improvements' are used. Not all improvements are related to performance, only classify a message as 'Yes' if it specifically mentions enhancements related to execution time, memory usage, or resource efficiency. Avoid making assumptions based on ambiguous terms. You should have high confidence in classifying a message as 'Yes' based on careful examination of the information provided in the commit message.

If the commit message contain multiple intentions, where at least one of those intentions includes a performance improvement, classify the entire commit as 'Yes'.
You will only respond with the predefined category. Do not include the word 'Category'. Do not provide explanations or notes.

Commit message: ```{commit_message}```
[/INST] Model_answer:  </s>'''

    prompts = [
        prompt_template.format(
            commit_message=record['commit_message']) for record in data]

    # Generate Mistral predictions
    sampling_params = SamplingParams(temperature=0.1, top_p=0.95, max_tokens=5)
    outputs = llm.generate(prompts, sampling_params)
    
    # Clean and map the Mistral predictions
    mistral_classifications = [output.outputs[0].text.strip() for output in outputs]
    cleaned_predictions = [clean_prediction(pred) for pred in mistral_classifications]
    
        # Save the results to a CSV file
    df = pd.DataFrame(data)
    df['Mistral_classification'] = mistral_classifications
    df['Mistral_Target'] = cleaned_predictions
    return df


Version 2: 

In [4]:
def process_and_evaluate(test_data_path, output_csv_path):
    # Load the true labels and data from the .jsonl file
    data = []
    y_true = []
    with open(test_data_path, 'r') as f:
        for line in f:
            record = json.loads(line)
            data.append(record)  # Store each record for later
            y_true.append(record['target'])  # Assuming 'target' is the true label field

    # Initialize the Mistral LLM
    llm = LLM(model="/home/akazad/Mistral-7B-Instruct-v0.2-AWQ/", tensor_parallel_size=2, gpu_memory_utilization=0.8, enforce_eager=True, quantization="awq", dtype="auto")
    #llm = LLM(model="/home/akazad/Mistral-7B-Instruct-v0.2/", tensor_parallel_size=2, gpu_memory_utilization=0.8, enforce_eager=True, quantization="awq", dtype="auto")

    # Prepare the prompts
    prompt_template = ''' <s> [INST] You are an analytical tool specialized in processing and classifying GitHub commit messages along with their associated code diffs. Your task is to assess the developer's intent in a given commit message and code diff, and categorize it into one of the following predefined categories based on its content:
                      
'Yes': A commit message and code diff that explicitly mention or demonstrate performance improvement or optimization, specifically in terms of execution time or resource utilization. The message or diff should clearly indicate actions that made the code run faster or more efficiently, use less memory, or more efficiently utilize system resources. This includes optimizations like replacing inefficient code patterns that are known to kill performance even if the message does not use the words 'improvement' or 'performance' explicitly. The diff may show changes such as algorithmic improvements, memory management enhancements, reduced computational complexity, or better resource utilization.

'No': A commit message and code diff that do not pertain to performance enhancements. This includes changes related to testing, documentation, performance profiling/monitoring/debugging/analysis, and bug/error/crash fixes that don't explicitly mention performance improvement of the application itself. Also, code refactoring or feature addition without explicit performance optimization, and mentions of necessary, speculative, or potential performance enhancements without concrete evidence or results. Any message or diff that is irrelevant, unclear, ambiguous, or does not provide enough context to determine its intent should also be classified as 'No'. 

Additionally, pay close attention to the context in which terms like 'performance', 'improve', or 'improvements' are used. Not all improvements are related to performance—only classify a message as 'Yes' if it specifically mentions enhancements related to execution time, memory usage, or resource efficiency. Avoid making assumptions based on ambiguous terms. You should have high confidence in classifying a message as 'Yes' based on careful examination of the information provided in the commit message and code diff.

If the commit message and code diff contain multiple intentions, where at least one of those intentions includes a performance improvement, classify the entire commit as 'Yes'.
You will only respond with the predefined category. Do not include the word 'Category'. Do not provide explanations or notes.

Commit message: ```{commit_message}```
Code diff: ```{code_diff}```
[/INST] Model_answer:  </s>'''

    prompts = [
        prompt_template.format(
            commit_message=record['commit_message'], code_diff=record['code_diff']) for record in data]

    # Generate Mistral predictions
    sampling_params = SamplingParams(temperature=0.1, top_p=0.95, max_tokens=5)
    outputs = llm.generate(prompts, sampling_params)
    
    # Clean and map the Mistral predictions
    mistral_classifications = [output.outputs[0].text.strip() for output in outputs]
    cleaned_predictions = [clean_prediction(pred) for pred in mistral_classifications]
    
        # Save the results to a CSV file
    df = pd.DataFrame(data)
    df['Mistral_classification'] = mistral_classifications
    df['Mistral_Target'] = cleaned_predictions
    return df


In [6]:
clean_prediction

<function __main__.clean_prediction(text)>

In [ ]:

    # Save the results to a CSV file
    df = pd.DataFrame(data)
    df['Mistral_classification'] = mistral_classifications
    df['Mistral_Target'] = cleaned_predictions

    # Only save necessary fields
    results_df = df[['commit_url', 'commit_message', 'code_diff', 'Mistral_Target']].copy()
    results_df.to_csv(output_csv_path, index=False)

    # Convert y_true to a numpy array for consistency
    y_true = np.array(y_true)

    # Convert cleaned predictions ('Yes' -> 1, 'No' -> 0)
    mistral_numeric_predictions = np.array([1 if pred == 'Yes' else 0 for pred in cleaned_predictions if pred is not None])

    # Handle case when there are no predictions or only a single class
    if len(np.unique(mistral_numeric_predictions)) == 0:
        print("No predictions were made.")
        return

    # Calculate F1 Score and other metrics
    report = classification_report(
        y_true[:len(mistral_numeric_predictions)],
        mistral_numeric_predictions,
        target_names=['No', 'Yes'],
        labels=[0, 1],  # Specify the labels explicitly
        zero_division=0  # Avoid zero division issues
    )

    # Write the classification report to the same file
    with open(output_csv_path, 'a') as file:
        file.write("\nClassification Report:\n")
        file.write(report)

    print("All data and results have been saved to", output_csv_path)

In [5]:
# Example usage
test_data_path = '/home/akazad/gt_final.jsonl'  # Path to your test data
output_csv_path = 'Mistral_goldmd_results.csv'  # Path to save the results
result = process_and_evaluate(test_data_path, output_csv_path)

INFO 10-23 15:32:00 awq_marlin.py:101] Detected that the model can run with awq_marlin, however you specified quantization=awq explicitly, so forcing awq. Use quantization=awq_marlin for faster inference
WARNING 10-23 15:32:00 config.py:321] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 10-23 15:32:00 config.py:905] Defaulting to use mp for distributed inference
WARNING 10-23 15:32:00 config.py:395] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 10-23 15:32:00 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/home/akazad/Mistral-7B-Instruct-v0.2-AWQ/', speculative_config=None, tokenizer='/home/akazad/Mistral-7B-Instruct-v0.2-AWQ/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dty

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(VllmWorkerProcess pid=2862315) INFO 10-23 15:32:02 model_runner.py:1067] Loading model weights took 1.9429 GB
INFO 10-23 15:32:02 model_runner.py:1067] Loading model weights took 1.9429 GB
INFO 10-23 15:32:07 distributed_gpu_executor.py:57] # GPU blocks: 14387, # CPU blocks: 4096
INFO 10-23 15:32:07 distributed_gpu_executor.py:61] Maximum concurrency for 32768 tokens per request: 7.02x


Processed prompts: 100%|██████████| 276/276 [00:34<00:00,  7.95it/s, est. speed input: 6134.14 toks/s, output: 39.74 toks/s]

INFO 10-23 15:32:45 multiproc_worker_utils.py:133] Terminating local vLLM worker processes
(VllmWorkerProcess pid=2862315) INFO 10-23 15:32:45 multiproc_worker_utils.py:240] Worker exiting


In [6]:
result.columns

Index(['commit_url', 'commit_message', 'code_diff',
       'commit_message_token_length', 'code_token_length',
       'combined_token_length', 'target', 'Mistral_classification',
       'Mistral_Target'],
      dtype='object')

In [7]:
result

,commit_url,commit_message,code_diff,commit_message_token_length,code_token_length,combined_token_length,target,Mistral_classification,Mistral_Target
0,https://github.com/qbittorrent/qBittorrent/com...,Change MixedModeAlgorithm default to TCP. Clos...,"@@ -304,7 +304,7 @@ Session::Session(QObject *...",55,287,342,1,No.\n\nEx,No
1,https://github.com/fzi-forschungszentrum-infor...,Further increase wait times for shove tree tes...,"@@ -192,13 +192,13 @@ class TestShoveTree(unit...",9,318,327,0,No.\n\nEx,No
2,https://github.com/tonlabs/TVM-Compiler/commit...,ValueTracking: Small cleanup in ComputeNumSign...,"@@ -1768,7 +1768,7 @@ unsigned ComputeNumSignB...",43,414,457,0,No.\n\nEx,No
3,https://github.com/git/git/commit/a6eec1263883...,upload-pack: drop lookup-before-parse optimiza...,"@@ -327,9 +327,7 @@ static int got_sha1(char *...",189,148,337,1,No.\n\nEx,No
4,https://github.com/systemd/systemd/commit/66a6...,sd-dhcp-client: --omg-optimized\n\nPassing the...,"@@ -37,8 +37,7 @@ int dhcp_network_bind_raw_so...",183,218,401,1,Yes.,Yes
...,...,...,...,...,...,...,...,...,...
271,https://github.com/hfinkel/llvm-project-cxxjit...,Output a very high-precision number llvm-svn:...,"@@ -80,7 +80,7 @@ static inline std::string it...",17,91,108,0,No.\n\nEx,No
272,https://github.com/intel/cm-compiler/commit/d1...,Handle functions with multiple exit blocks pro...,"@@ -526,6 +526,9 @@ bool GVNPRE::runOnFunction...",17,383,400,0,No.\n\nEx,No
273,https://github.com/ilstam/FF-Multi-Converter/c...,use textwrap.fill() from stdlib to wrap long l...,"@@ -30,7 +30,7 @@ import sys\n import re\n imp...",13,452,465,0,No.\n\nEx,No
274,https://github.com/NYRDS/remixed-dungeon/commi...,33% chance for lich to jump on hit,"@@ -199,7 +199,11 @@ public class Lich extends...",9,141,150,0,No.\n\nEx,No


In [8]:
result['Mistral_Target'].value_counts()

Mistral_Target
No     213
Yes     63
Name: count, dtype: int64

In [9]:
import pandas as pd
from sklearn.metrics import classification_report


df = result
# Assuming df is your DataFrame
# Create a new column 'target_prediction' with 1 for 'Yes' and 0 for 'No'
df['predicted'] = df['Mistral_Target'].map({'Yes': 1, 'No': 0})

In [10]:
df.value_counts('predicted')

predicted
0    213
1     63
Name: count, dtype: int64

In [12]:
# Extract 'target' and 'target_prediction' columns as lists
import csv 

y_true = df['target'].tolist()
y_pred = df['predicted'].tolist()
commit_urls = df['commit_url'].tolist()

assert len(y_true) == len(y_pred)
# Write commit_message, code_diff, target, and predicted values to a CSV file
with open('Mistral_goldmd_results_latest.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    # Write header
    csvwriter.writerow(['commit_url', 'original_target', 'predicted'])
    
    # Write data rows
    for i in range(len(df)):
        csvwriter.writerow([commit_urls[i], y_true[i], y_pred[i]])
# Generate the classification report
report = classification_report(y_true, y_pred)

# Print the classification report
print(report)

with open('Mistral_goldmd_classification_report.txt', 'w') as file:  # Append the classification report
    file.write("\nClassification Report:\n")
    file.write(report) 
    


              precision    recall  f1-score   support

           0       0.64      0.99      0.78       138
           1       0.98      0.45      0.62       138

    accuracy                           0.72       276
   macro avg       0.81      0.72      0.70       276
weighted avg       0.81      0.72      0.70       276



In [ ]:
import pandas as pd
from vllm import LLM, SamplingParams
from sklearn.metrics import classification_report
import os
import re
import json
import numpy as np








In [ ]:
import pandas as pd
from vllm import LLM, SamplingParams
import os

def process_batches(df, batch_size, output_csv_path):
    # Determine the starting batch index in case of a restart
    if os.path.exists(output_csv_path):
        existing_df = pd.read_csv(output_csv_path,engine='python').dropna()
        processed_entries = len(existing_df)
        start_batch_idx = processed_entries // batch_size
        print(f"Resuming from batch {start_batch_idx}.")
    else:
        start_batch_idx = 0

    llm = LLM(model="/home/akazad/Mistral-7B-Instruct-v0.2-AWQ/",tensor_parallel_size=2,gpu_memory_utilization=0.8,enforce_eager=True,
              quantization="awq", dtype="auto")
    
    num_batches = (len(df) + batch_size - 1) // batch_size  # Calculate total number of batches

    for batch_idx in range(start_batch_idx, num_batches):
        start_idx = batch_idx * batch_size
        end_idx = start_idx + batch_size
        batch_df = df.iloc[start_idx:end_idx]

        prompt_template = ''' <s> [INST] You are an analytical tool specialized in processing and classifying GitHub commit messages along with their associated code diffs. Your task is to assess the developer's intent in a given commit message and code diff, and categorize it into one of the following predefined categories based on its content:
                      
'Yes': A commit message and code diff that explicitly mention or demonstrate performance improvement or optimization, specifically in terms of execution time or resource utilization. The message or diff should clearly indicate actions that made the code run faster or more efficiently, use less memory, or more efficiently utilize system resources. This includes optimizations like replacing inefficient code patterns that are known to kill performance even if the message does not use the words 'improvement' or 'performance' explicitly. The diff may show changes such as algorithmic improvements, memory management enhancements, reduced computational complexity, or better resource utilization.

'No': A commit message and code diff that do not pertain to performance enhancements. This includes changes related to testing, documentation, performance profiling/monitoring/debugging/analysis, and bug/error/crash fixes that don't explicitly mention performance improvement of the application itself. Also, code refactoring or feature addition without explicit performance optimization, and mentions of necessary, speculative, or potential performance enhancements without concrete evidence or results. Any message or diff that is irrelevant, unclear, ambiguous, or does not provide enough context to determine its intent should also be classified as 'No'. Additionally, pay close attention to the context in which terms like 'performance', 'improve', or 'improvements' are used. Not all improvements are related to performance—only classify a message as 'Yes' if it specifically mentions enhancements related to execution time, memory usage, or resource efficiency. Avoid making assumptions based on ambiguous terms. You should have high confidence in classifying a message as 'Yes' based on careful examination of the information provided in the commit message and code diff.

If the commit message and code diff contain multiple intentions, where at least one of those intentions includes a performance improvement, classify the entire commit as 'Yes'.
You will only respond with the predefined category. Do not include the word 'Category'. Do not provide explanations or notes.

Commit message: ```{commit_message}```
Code diff: ```{code_diff}```
[/INST] Model_answer:  </s>'''


        prompts = [
            prompt_template.format(
                commit_message=row['commit_message'],code_diff=row['diff']) for _, row in batch_df.iterrows()]

        sampling_params = SamplingParams(temperature=0.1, top_p=0.95, max_tokens=5)
        outputs = llm.generate(prompts, sampling_params)
        batch_df['Mistral_classification']=[output.outputs[0].text for output in outputs]
        results_df=batch_df[['commit_url','commit_message', 'diff','nloc', 'n_added_lines', 'n_deleted_lines','Mistral_classification']].copy()
        if batch_idx == 0 and not os.path.exists(output_csv_path):
            results_df.to_csv(output_csv_path, mode='w', index=False)
        else:
            results_df.to_csv(output_csv_path, mode='a', header=False, index=False)

        print(f"Batch {batch_idx} processed and saved.")

    print(f"All data has been saved to {output_csv_path}")

# Example usage of the function
output_csv_path = 'Data/Mistral_classification_with_patch_java_1L.csv'
process_batches(cpp_filtered, 10000, output_csv_path)


## generate CodeBERT-KD report

In [1]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import time
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import json

# # Load tokenizer and model # LATEST CodeBERT KD Model
# tokenizer = RobertaTokenizer.from_pretrained('ManojAlexender/second_Base_version_of_codebert_with_commit_and_diff')
# model = RobertaForSequenceClassification.from_pretrained('ManojAlexender/second_Base_version_of_codebert_with_commit_and_diff')

# Load tokenizer and model # OLD KD MODEL
tokenizer = RobertaTokenizer.from_pretrained('ManojAlexender/second_Base_version_of_codebert_with_commit_and_diff')
model = RobertaForSequenceClassification.from_pretrained('ManojAlexender/second_Base_version_of_codebert_with_commit_and_diff')


# Load the data from the .jsonl file
data = []
y_true = []
commit_messages = []  # To store commit messages
code_diffs = []  # To store code diffs
commit_urls = []

with open('gt_final.jsonl', 'r') as f:
    for line in f:
        record = json.loads(line)
        # Concatenate 'commit_message' and 'code_diff' fields for the model
        commit_message = '"'+ record['commit_message'].replace('\n', ' ') + '"'
        code_diff = record['code_diff']
        commit_url = record['commit_url']
        #data.append(commit_message + ' ' + code_diff) # commit+diff
        data.append(commit_message) #commitonly
        # Append the target field to y_true
        y_true.append(record['target'])
        # Store the commit_message and code_diff for later writing
        commit_messages.append(commit_message)
        code_diffs.append(code_diff)
        commit_urls.append(commit_url)
        

# Convert y_true to a numpy array for later use
y_true = np.array(y_true)

# Function to process data instance by instance and calculate predictions
def predict_instance_by_instance(model, tokenizer, data):
    model.to('cuda')
    model.eval()
    all_predictions = []

    for instance in data:
        inputs = tokenizer(instance, return_tensors='pt', padding=True, truncation=True, max_length=512)
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()
        all_predictions.append(prediction)

    return all_predictions

# Run prediction once and generate the classification report
predictions = predict_instance_by_instance(model, tokenizer, data)

# Convert predictions to numpy array and adjust labels if necessary
predictions = np.array(predictions)
predictions[predictions == 2] = 0  # Adjust according to your label scheme if needed

In [4]:
import csv

# Verify that the predictions array has the same length as the original data
assert len(predictions) == len(data), "Length of predictions does not match the length of the original data."

# Write commit_message, code_diff, target, and predicted values to a CSV file
with open('RoBERTa_KD_goldm_results.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    # Write header
    csvwriter.writerow(['commit_url', 'original_target', 'predicted'])
    
    # Write data rows
    for i in range(len(data)):
        csvwriter.writerow([commit_urls[i], y_true[i], predictions[i]])

In [5]:
# Generate classification report and write it to the same file
report = classification_report(y_true, predictions)
with open('RoBERTa_KD_goldm_classification_report.txt', 'w') as file:  # Append the classification report
    file.write("\nClassification Report:\n")
    file.write(report) 

## generate RoBERTa-KD report

In [4]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import time
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import json

# # Load tokenizer and model # LATEST CodeBERT KD Model
# tokenizer = RobertaTokenizer.from_pretrained('ManojAlexender/second_Base_version_of_codebert_with_commit_and_diff')
# model = RobertaForSequenceClassification.from_pretrained('ManojAlexender/second_Base_version_of_codebert_with_commit_and_diff')

# Load tokenizer and model # OLD KD MODEL
tokenizer = RobertaTokenizer.from_pretrained('ManojAlexender/final_roberta_with_new_400k_plus_37k_Best')
model = RobertaForSequenceClassification.from_pretrained('ManojAlexender/final_roberta_with_new_400k_plus_37k_Best')


# Load the data from the .jsonl file
data = []
y_true = []
commit_messages = []  # To store commit messages
code_diffs = []  # To store code diffs
commit_urls = []

with open('gt_final.jsonl', 'r') as f:
    for line in f:
        record = json.loads(line)
        # Concatenate 'commit_message' and 'code_diff' fields for the model
        commit_message = '"'+ record['commit_message'].replace('\n', ' ') + '"'
        code_diff = record['code_diff']
        commit_url = record['commit_url']
        data.append(commit_message + ' ' + code_diff) # commit+diff
        #data.append(commit_message) #commitonly
        # Append the target field to y_true
        y_true.append(record['target'])
        # Store the commit_message and code_diff for later writing
        commit_messages.append(commit_message)
        code_diffs.append(code_diff)
        commit_urls.append(commit_url)
        

# Convert y_true to a numpy array for later use
y_true = np.array(y_true)

# Function to process data instance by instance and calculate predictions
def predict_instance_by_instance(model, tokenizer, data):
    model.to('cuda')
    model.eval()
    all_predictions = []

    for instance in data:
        inputs = tokenizer(instance, return_tensors='pt', padding=True, truncation=True, max_length=512)
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()
        all_predictions.append(prediction)

    return all_predictions

# Run prediction once and generate the classification report
predictions = predict_instance_by_instance(model, tokenizer, data)

# Convert predictions to numpy array and adjust labels if necessary
predictions = np.array(predictions)
predictions[predictions == 2] = 0  # Adjust according to your label scheme if needed

In [5]:
import csv

# Verify that the predictions array has the same length as the original data
assert len(predictions) == len(data), "Length of predictions does not match the length of the original data."

# Write commit_message, code_diff, target, and predicted values to a CSV file
with open('RoBERTa_KD_goldmd_results.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    # Write header
    csvwriter.writerow(['commit_url', 'original_target', 'predicted'])
    
    # Write data rows
    for i in range(len(data)):
        csvwriter.writerow([commit_urls[i], y_true[i], predictions[i]])

In [6]:
# Generate classification report and write it to the same file
report = classification_report(y_true, predictions)
with open('RoBERTa_KD_goldmd_classification_report.txt', 'w') as file:  # Append the classification report
    file.write("\nClassification Report:\n")
    file.write(report) 